In [14]:
import faiss
import sys

modules = '/home/dbcloud/ym/CSPG/ANNS/modules'

if modules not in sys.path:
  sys.path.append(modules)

from binary_io import *
import anns

In [15]:
dataset = 'gud'

read_ways = 'fvecs_read'

base = "/home/dbcloud/ym/CSPG/experiment/distribution/data/ubase.fvecs"

query = "/home/dbcloud/ym/CSPG/experiment/distribution/data/uquery.fvecs"


gt = "/home/dbcloud/ym/CSPG/experiment/distribution/data/ugt.ivecs"

output = '/home/dbcloud/ym/CSPG/experiment/output/distribution/'

In [16]:
def test_faiss_ivf(base, query, gt, rw, dataset):

  if rw == 'fbin_read':
    base = fbin_read(base)
    query = fbin_read(query)
  else:
    base = fvecs_read(base)
    query = fvecs_read(query)

  if dataset == 'deep1m':
    gt = ibin_read(gt)
  else:
    gt = ivecs_read(gt)

  nb, d = base.shape
  nq, ngt = gt.shape
  gt = gt.flatten().tolist()

  k = 10
  threads = 24
  faiss.omp_set_num_threads(threads)

  nlist = 1204

  quantizer = faiss.IndexFlatL2(d) 
  index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
  # Train the index
  index.train(base)
  # Add the base vectors to the index
  index.add(base)

  print(dataset, nb, d, nq, nlist, k)

  out = open(output + dataset + '_ivf.csv', 'w')

  tm = anns.STimer()

  print('num_queries,nlist,nprobe,query_time,recall', file=out)
  for npb in range(10, 200+1, 10):
    index.nprobe = npb
    tm.reset()
    tm.start()
    D, I = index.search(query, k)
    tm.stop()
    query_time = tm.get_time()
    recall = anns.get_recall(k, ngt, gt, I)
    print(f"{nq},{nlist},{npb},{query_time},{recall}", file=out)

  out.close()

In [17]:
# for ds, rw, base, query, gt in zip(dataset, read_ways, bases, queries, gts):
#   test_faiss_ivf(base, query, gt, rw, ds)
test_faiss_ivf(base, query, gt, read_ways, dataset)

gud 1000000 96 1000 1204 10
